# Look at instances of top terms in corpora

In [1]:
# Load corpora/splits
import pickle
import os

data_dirpath = '/home/mamille3/hegemonic_hate/data'
# splits_fpath = os.path.join(data_dirpath, 'combined_identity_splits_kennedy2020+sbic+hatexplain+civilcomments_0.3hate.pkl')
split_type = 'power'
splits_fpath = os.path.join(data_dirpath, f'combined_{split_type}_kennedy2020+sbic+hatexplain+civilcomments_0.3hate.pkl')
with open(splits_fpath, 'rb') as f:
    corpora = pickle.load(f)
    
corpora.keys()

dict_keys([('hegemonic',), ('marginalized',)])

In [21]:
import pandas as pd

split = ('marginalized',)
data = pd.concat([corpora[split]['train'], corpora[split]['test']]).query('hate')
term = 'my'
data['text_lower'] = data.text.str.lower()
selected = data[data.text_lower.str.contains(r'\b{}\b'.format(term))]
selected.sample(10).loc[:, ['text_lower', 'hate']]

,text_lower,hate
text_id,,
27525,@yakerriah @nomnomnyandu fuck you you disgusting dung eating african. we fucking wash. talk like this to my face you backwards cunt. fuck you. get the fuck out of europe,True
3536,i should have let an old man white man work on my car from the beginning instead of letting these niggers and spics play under my hood,True
5626464,"one of my comments has disappeared already. to be more blunt, all black men aren't criminals, and all black mothers aren't saints.",True
5835,tht bisexual person who said dyke apologized for it and deleted the tweet and all ppl are replying under the apology is stream my fancam and __ stans follow me like some ppl are so generic and unfunny it hurts,True
3783,going to start exercising my constitutional right call people dumb faggots in public,True
5710,"oh look, niggers being savage. imagine my shock.",True
24812,what are my views on slavery? works for me...,True
40240,"""why people think gay marriage is okay is beyond me. sorry i don't want my future son seeing 2 fags walking down the street holding hands an",True
42761,"sorry , that must be my black coming out.",True


# Prepare corpora for SAGE
Then run it at ~/SAGE/py-sage (repo fork at https://github.com/michaelmilleryoder/SAGE/tree/master/py-sage)

In [11]:
# Load corpora/splits
import pickle
import os
import pandas as pd
from tqdm.notebook import tqdm
import spacy
import re

nlp = spacy.load('en_core_web_sm', exclude=['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner'])
nlp.add_pipe('sentencizer')

data_dirpath = '/home/mamille3/hegemonic_hate/data'
# splits_fpath = os.path.join(data_dirpath, 'combined_identity_splits_kennedy2020+sbic+hatexplain+civilcomments_0.3hate.pkl')

split_types = ['identity', 'categories', 'power']
for split_type in split_types:
    splits_fpath = os.path.join(data_dirpath, f'combined_{split_type}_kennedy2020+sbic+hatexplain+civilcomments_0.3hate.pkl')
    with open(splits_fpath, 'rb') as f:
        corpora = pickle.load(f)

    texts = {}

    split_transform = {
        ('race/ethnicity',): 'race/ethnicity',
        ('religion',): 'religion',
        ('gender', 'sexuality'): 'gender/sexuality',
        ('hegemonic',): 'hegemonic',
        ('marginalized',): 'marginalized',
    }

    # for split in list(corpora.keys())[:1]:
    for split in tqdm(corpora):
        data = pd.concat([corpora[split]['train'], corpora[split]['test']])
        hate = data.query('hate')
        texts[split] = hate.text.tolist()

        # Process, save out data
        processed = []
        inp = texts[split]
        # for doc in tqdm(nlp.pipe(inp), total=len(inp)):
        for doc in nlp.pipe(inp):
            sents = [' '.join([tok.text for tok in sent]).strip().lower() for sent in doc.sents]
            processed.extend(sents)

        # Save out
        out_dirpath = os.path.join(data_dirpath, split_type)
        if not os.path.exists(out_dirpath):
            os.mkdir(out_dirpath)
        outpath = os.path.join(out_dirpath, f'{re.sub(r"[ /,]", "_", split_transform.get(split, split))}_sents.txt')
        with open(outpath, 'w') as f:
            for sent in processed:
                f.write(f'{sent}\n')

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

# Read, format SAGE output

In [24]:
import os
import pandas as pd
from IPython.display import display

split_types = ['identity', 'categories', 'power']
for split_type in split_types:
    n_words = 10
    vocab_size = 2000
    sage_outpath = f'/home/mamille3/SAGE/py-sage/output/{split_type}_{n_words}words_{vocab_size}vocab.csv'
    results = pd.read_csv(sage_outpath, sep='\t')
    # results['split'] = results.source.str.slice(len(f'input/{split_type}/'), -1*(len('_sents.txt')))
    results['split'] = results.source.map(lambda x: os.path.basename(x)[:-1*(len('_sents.txt'))])
    # results

    pd.set_option('display.max_colwidth', None)
    top_terms = results.groupby('split').agg({'word': ', '.join})
    if split_type == 'identity':
        top_terms.drop('people_of_color', inplace=True)
        top_terms.index = top_terms.index.str.replace('_', ' ')
        top_terms.index = top_terms.index.str.capitalize()
    elif split_type == 'categories':
        top_terms.index = top_terms.index.str.replace('_', '/')
    top_terms.index.name = split_type.capitalize()
    top_terms.columns = ['Top terms']
    if split_type == 'identity':
        top_terms = top_terms.rename(index={'Lgbtq+ people': 'LGBTQ+ people', 'Muslims and arabic middle eastern people': 'Muslims, Arabs'})
    display(top_terms)

,Top terms
Identity,
Asian people,"ching, chong, chinese, china, asian, japan, japanese, chink, korea, gook"
Black people,"niggas, black, nigga, nfl, unarmed, africa, monkeys, negro, coon, cop"
Christians,"priest, christianity, catholics, jesus, catholic, bible, christians, christian, christ, priests"
Jews,"jewish, oven, zionist, gas, holocaust, jews, israel, jew, hitler, koran"
Latinx people,"beaner, latino, mexico, latinos, latin, spic, mexicans, mexican, beaners, wetback"
LGBTQ+ people,"homosexual, homosexuality, transgendered, queers, homosexuals, transgender, gay, heterosexual, fag, queer"
Men,"male, penis, incel, dollar, men, man, bisexual, males, cucks, gender"
"Muslims, Arabs","islamic, islam, muslim, isis, goat, iran, terrorist, bomb, islamophobia, arab"
White people,"supremacist, nationalists, supremacists, white, brothers, virgin, devil, nazi, chick, neo"


,Top terms
Categories,
gender/sexuality,"dyke, transgender, bathroom, queer, queers, lesbian, trannies, fag, trans, dykes"
race/ethnicity,"black, blacks, chinese, chicago, blm, asian, races, nationalists, whitey, asians"
religion,"priest, catholic, catholics, ash, christians, chamber, bishops, santa, christianity, christian"


,Top terms
Power,
hegemonic,"priest, catholic, priests, jesus, catholics, bible, supremacists, virgin, christian, christ"
marginalized,"black, nigger, muslims, muslim, blacks, she, her, women, woman, my"


In [20]:
latex = top_terms.style.to_latex(hrules=True, label=f'{split_type}_sage', caption=f'Most representative terms in corpora divided by target identity {split_type} from SAGE', environment='table*')
latex = latex.replace(' & Top terms', f'{split_type.capitalize()} & Top terms').replace(f'{split_type.capitalize()} &  \\\\\n', '').replace(' people', '').replace('hegemonic', 'dominant')
print(latex)

\begin{table*}
\caption{Most representative terms in corpora divided by target identity power from SAGE}
\label{power_sage}
\begin{tabular}{ll}
\toprule
Power & Top terms \\
\midrule
hegemonic & catholic, jesus, supremacists, christian, church, boys, christianity, boy, supremacist, mudshark \\
marginalized & black, nigger, muslims, muslim, blacks, she, her, women, woman, my \\
\bottomrule
\end{tabular}
\end{table*}

